#### write_based_data
True: Decide voltage based on average of real data<br>
False: Decide voltage based on average fitting model ->실제로는 가지도 못하는 resistance를 다루게 될 수 있음

원래 이론적으로는 target resistance도 결정한다, 그런데 target resistance를 결정 할 때 어쩔 수 없이 시그모이드함수가 들어간다.<br>
즉 write_based_data=True여도 실제로 generate 되는 resistance는 시그모이드와 함께 하기 때문에 실제로 generate 되는 resistance는 같다. <br>
음..쉽게 말하면 write_based_data가 True이든 False이든 결과에는 영향이 없다! 이 코드에서는 실질적인 변화를 위한 것이라기보다, data average와 fitting model average를 비교하기 위해 놓았다.<br>
(True로 놓으면 target resistance를 전자에서 가져오고,False로 놓으면 후자에서 가져온다).<br>

voltage를 어디서 결정할지도 사실 여기서 결정해야하지만 voltage는 미리 결정해놓았다고 가정한다.<br>
voltage를 fitting model에서 만들면 위에 언급한 문제가 발생한다. 즉 fitting모델은 특정구역에서 일반적 오차를 가져오는데 그 오차 때문에 실제로는 쓰기 힘든 resistance를 쓸 수 있다고 생각 하게 될 수도 있다. <br>
그래서 voltage를 고를 모델은 조금 더 신중하는게 좋다(오늘자 노트 참고).

#### read_based_data
True: Decide reference resistance based on average of real data<br>
False: Decide reference resistance based on average fitting model

In [1]:
import numpy as np
from openpyxl import load_workbook
import matplotlib.pyplot as plt
import pandas as pd
def make_index(x,ref):
    index = float(x>=ref[0])
    for ref in ref[1:]:
        index += float(x>=ref)
    return index

def load_info(num_level,choice=0):
    wb = load_workbook(filename='level_info.xlsx')
    sheetname = str(num_level) + 'Levels'
    if (str(wb.sheetnames).find(sheetname) > -1) and (wb[sheetname]['A' + str(3 * choice + 1)] != None):
        print("I find the information of %dLevels"%num_level)
        ws = wb[sheetname]
        volt = np.array([cell.value for cell in tuple(ws.rows)[4 * choice]]).astype('float32')
        r = np.array([cell.value for cell in tuple(ws.rows)[4 * choice+1]]).astype('float32')
        if ws['A' + str(4 * choice + 2)] != None:
            r_std = np.array([cell.value for cell in tuple(ws.rows)[4 * choice + 2]]).astype('float32')
        if ws['A' + str(4 * choice + 3)] != None:
#             r_ref = np.array([cell.value for cell in tuple(ws.rows)[4 * choice + 3][1:]]).astype('float32')   #잘 read 되는지 확인해야함
            r_ref = (r[1:] + r[0:-1]) / 2
    else:
        print("There is no information  of %dLevels"%num_level)
        volt = np.array([simple_r for simple_r in range(1, num_level + 1)]).astype('float32')
        r = np.array([simple_r for simple_r in range(1, num_level + 1)]).astype('float32')
        r_std = np.ones([num_level], dtype='float32')
        r_ref = (r[1:] + r[0:-1]) / 2
    return volt,r,r_std,r_ref

In [64]:
def scheme_check(volt,r,r_std,r_ref,num_level=14,num_write=1000,fitting_model_average_choice=False,
                 write_based_data=False,read_based_data=False,cycle_variation=False,device_variation=False,onecell=False):
    log=np.zeros([num_level,num_write,5])  #level i를 j번째 쓸 때 얻게 되는 5개의 정보를 쓰게 될 행렬
    log_level=np.zeros([num_level,num_level]) #level i를 num_write번 쓸 때 각 레벨이 몇번 나오게 되나를 기록 할 행렬
    a=3.156e+06
    b=4.463
    c=2.713
    d=2.509e+04
    if fitting_model_average_choice==False:
        mean_a,mean_b,mean_c,mean_d=a,b,c,d
    else:
        mean_a,mean_b,mean_c,mean_d=a*0.9996,b*1.0647,c*1.0004,d*1.0368
    if write_based_data == False: #
        r = np.array([mean_a / (1 + np.exp(-mean_b * (x - mean_c))) + mean_d for x in volt])
        
    if read_based_data == False:
        temp = np.array([mean_a / (1 + np.exp(-mean_b * (x - mean_c))) + mean_d for x in volt])
        r_ref = (temp[1:] + temp[0:-1]) / 2
#         r_ref = [ 508998.68485452,734426.74492238,1024890.55948719,1275528.5558163,1457834.55645558,1644133.88666727,1829306.37527018,2089524.18914522,2396897.257, 2640884.09748843,2820836.3687569,2979827.79184652,3099313.97217619]
#         r_ref=[ 553682.99450246,  777899.98328473, 1059132.06399215, 1298052.23847886,1470012.01854126, 1645204.55824665, 1819367.46320471, 2065746.25205046,2360407.67442528, 2599835.48980322, 2781688.95203013, 2949620.22106922,3081267.30830828]

    if cycle_variation == True: #True면 위에서 결정한거 그대로 쓰면 된다.
        meanofstd=0.6*r_std
        stdofstd=0.5*meanofstd
    else:
        meanofstd=np.zeros(len(r_std))
        stdofstd=np.zeros(len(r_std))
    
    if device_variation == True:
        std_a=abs(a)*0.0698
        std_b=abs(b)*0.1824
        std_c=abs(c)*0.0354
        std_d=abs(d)*0.7708
    else:
        std_a,std_b,std_c,std_d=0,0,0,0
        stdofstd=np.zeros(len(r_std))

    print("fitting_model_average_choice: \n[mean_a,mean_b,mean_c,mean_d]=",[mean_a,mean_b,mean_c,mean_d])
    print("write_based_data:\nr=",r)
    print("read_based_data:\nr_ref=",r_ref)
    print("cycle_variation:\nmeanofstd=",meanofstd,'\nstdofstd=',stdofstd)
    print("device_variation:\nstdofmean=",[std_a,std_b,std_c,std_d],"\nstdofstd=",stdofstd)
    cell_index=1
    cell_dict={}
    x_volt=np.linspace(1.7,4,231)
    cell_dict['Average Cell']=mean_a/(1+np.exp(-mean_b*(x_volt-mean_c))) + mean_d
    if onecell==True:
        a_var = np.random.normal(loc=mean_a, scale=std_a)
        b_var = np.random.normal(loc=mean_b, scale=std_b)
        c_var = np.random.normal(loc=mean_c, scale=std_c)
        d_var = np.random.normal(loc=mean_d, scale=std_d)
        cell_dict['Cell'+str(cell_index)]=a_var/(1+np.exp(-b_var*(x_volt-c_var))) + d_var
    for i in range(num_level):
        volt_write = volt[i]
        for j in range(num_write):
            if onecell==False:
                a_var = np.random.normal(loc=mean_a, scale=std_a)
                b_var = np.random.normal(loc=mean_b, scale=std_b)
                c_var = np.random.normal(loc=mean_c, scale=std_c)
                d_var = np.random.normal(loc=mean_d, scale=std_d)
                cell_dict['Cell'+str(cell_index).rjust(3,'0')]=a_var/(1+np.exp(-b_var*(x_volt-c_var))) + d_var
                cell_index+=1
            mean_val = a_var/(1+np.exp(-b_var*(volt_write-c_var))) + d_var
            std_val = np.random.normal(loc=meanofstd[i],scale=stdofstd[i])
            std_val = std_val *(std_val>0) + meanofstd[i]* (std_val<=0)
            r_write=np.random.normal(loc=mean_val,scale=std_val)
            level_read=make_index(r_write,r_ref)
            log[i,j,0]=i
            log[i,j,1]=r[i]
            log[i,j,2]=r_write
            log[i,j,3]=level_read
            log[i,j,4]=(i==level_read)
            log_level[i,int(level_read)]+=1
        print(mean_val)
        print([log[i, j, 0:3],mean_val,log[i,j,3],np.sum(log[i,:,4])])
        print(log_level[i,:])
    col_index=[1.7+0.01*i for i in range(231)]
#     cell_function=pd.DataFrame(cell_dict, index=col_index)
#     ax=cell_function.plot()
#     ax.legend(bbox_to_anchor=(1.1, 1.05))
#     plt.show()
    print('finish')

In [9]:
volt,r,r_std,r_ref=load_info(num_level=15,choice=0)
print(volt)
volt,r,r_std,r_ref=load_info(num_level=15,choice=1)
print(volt)
volt,r,r_std,r_ref=load_info(num_level=15,choice=2)
print(volt)

I find the information of 15Levels
[1.77 2.2  2.35 2.45 2.53 2.6  2.66 2.72 2.78 2.84 2.91 2.99 3.09 3.25
 4.  ]
I find the information of 15Levels
[1.77 2.07 2.23 2.35 2.45 2.54 2.63 2.72 2.81 2.9  2.99 3.09 3.21 3.38
 3.78]
I find the information of 15Levels
[1.77 2.08 2.22 2.33 2.43 2.53 2.63 2.72 2.81 2.91 3.01 3.12 3.24 3.4
 4.  ]


In [22]:
scheme_check(volt,r,r_std,r_ref,14,1000,fitting_model_average_choice=False,
             write_based_data=False,read_based_data=False,cycle_variation=False,device_variation=False)

fitting_model_average_choice: 
[mean_a,mean_b,mean_c,mean_d]= [3156000.0, 4.463, 2.713, 25090.0]
write_based_data:
r= [ 456423.69687235  650942.29213256  904857.6744369  1213406.45354739
 1382698.02341033 1557326.01367219 1733083.1028211  1905651.82358832
 2225840.68051259 2494974.66833797 2704696.31126847 2858681.59279179
 3040558.84934666 3121975.7672699 ]
read_based_data:
r_ref= [553682.99450246, 777899.98328473, 1059132.06399215, 1298052.23847886, 1470012.01854126, 1645204.55824665, 1819367.46320471, 2065746.25205046, 2360407.67442528, 2599835.48980322, 2781688.95203013, 2949620.22106922, 3081267.30830828]
cycle_variation:
meanofstd= [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 
stdofstd= [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
device_variation:
stdofmean= [0, 0, 0, 0] 
stdofstd= [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[array([     0.        , 456423.69687235, 456423.69687235,      0.        ]), 1000.0]
[array([1.00000000e+00, 6.50942292e+05, 6.50942292e+05, 1.00000000e+00])

In [23]:
scheme_check(volt,r,r_std,r_ref,14,1000,fitting_model_average_choice=False,
             write_based_data=False,read_based_data=False,cycle_variation=False,device_variation=False)

fitting_model_average_choice: 
[mean_a,mean_b,mean_c,mean_d]= [3156000.0, 4.463, 2.713, 25090.0]
write_based_data:
r= [ 456423.69687235  650942.29213256  904857.6744369  1213406.45354739
 1382698.02341033 1557326.01367219 1733083.1028211  1905651.82358832
 2225840.68051259 2494974.66833797 2704696.31126847 2858681.59279179
 3040558.84934666 3121975.7672699 ]
read_based_data:
r_ref= [553682.99450246, 777899.98328473, 1059132.06399215, 1298052.23847886, 1470012.01854126, 1645204.55824665, 1819367.46320471, 2065746.25205046, 2360407.67442528, 2599835.48980322, 2781688.95203013, 2949620.22106922, 3081267.30830828]
cycle_variation:
meanofstd= [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 
stdofstd= [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
device_variation:
stdofmean= [0, 0, 0, 0] 
stdofstd= [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[array([     0.        , 456423.69687235, 456423.69687235,      0.        ]), 1000.0]
[array([1.00000000e+00, 6.50942292e+05, 6.50942292e+05, 1.00000000e+00])

위 두개의 예는 fitting_model_average_choice만 다릅니다, 이를 통해 mean of a,b,c,d를 실제 데이터 mean에 기반한(false)와 실제 데이터 fitting model의 평균에 기반한(True)의 차이가 그렇게 크지 않다는 것을 알 수 있습니다.(즉 서로 섞어써도 레벨을 인식하는데 무리가 없다) 
레벨이 올라가면 저 조금의 차이도 인식에 차이를 가져오긴 하겠지만, variation을 고려한다면 그렇게 significant한 영향을 줄 것 같지는 않습니다.

주의:write_based_data=False일 때만(최소한 read_based_data=False여야) fitting_model_average_choice의 true/false가 의미가 있습니다. fitting model의 평균을 뭘로 할 것이냐를 결정하는데, 만약 fitting model을 안쓰면 의미가 없겠죠?

In [40]:
scheme_check(volt,r,r_std,r_ref,14,1000,fitting_model_average_choice=False,
             write_based_data=True,read_based_data=False,cycle_variation=False,device_variation=False)

fitting_model_average_choice: 
[mean_a,mean_b,mean_c,mean_d]= [3154737.6, 4.7517561, 2.7140852, 26013.311999999998]
write_based_data:
r= [ 408093.4   647025.06  968410.6  1275442.1  1394173.8  1586317.6
 1773990.4  1923664.4  2242287.5  2431918.8  2699296.   2782773.5
 2998088.2  3142365.2 ]
read_based_data:
r_ref= [ 508998.68485452  734426.74492238 1024890.55948719 1275528.5558163
 1457834.55645558 1644133.88666727 1829306.37527018 2089524.18914522
 2396897.25713225 2640884.09748843 2820836.3687569  2979827.79184652
 3099313.97217619]
cycle_variation:
meanofstd= [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 
stdofstd= [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
device_variation:
stdofmean= [0, 0, 0, 0] 
stdofstd= [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[array([     0.        , 408093.40625   , 412921.21487398,      0.        ]), 1000.0]
[array([1.00000000e+00, 6.47025062e+05, 6.05076155e+05, 1.00000000e+00]), 1000.0]
[array([2.00000000e+00, 9.68410625e+05, 8.63777335e+05, 2.00000000

In [41]:
scheme_check(volt,r,r_std,r_ref,14,1000,fitting_model_average_choice=False,
             write_based_data=False,read_based_data=True,cycle_variation=False,device_variation=False)

fitting_model_average_choice: 
[mean_a,mean_b,mean_c,mean_d]= [3156000.0, 4.463, 2.713, 25090.0]
write_based_data:
r= [ 456423.69687235  650942.29213256  904857.6744369  1213406.45354739
 1382698.02341033 1557326.01367219 1733083.1028211  1905651.82358832
 2225840.68051259 2494974.66833797 2704696.31126847 2858681.59279179
 3040558.84934666 3121975.7672699 ]
read_based_data:
r_ref= [ 527559.25  807717.9  1121926.4  1334808.   1490245.8  1680154.
 1848827.4  2082976.   2337103.   2565607.5  2741034.8  2890431.
 3070226.8 ]
cycle_variation:
meanofstd= [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 
stdofstd= [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
device_variation:
stdofmean= [0, 0, 0, 0] 
stdofstd= [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[array([     0.        , 456423.69687235, 456423.69687235,      0.        ]), 1000.0]
[array([1.00000000e+00, 6.50942292e+05, 6.50942292e+05, 1.00000000e+00]), 1000.0]
[array([2.00000000e+00, 9.04857674e+05, 9.04857674e+05, 2.00000000e+00]), 1000.0

위 두개의 예는 write_based_data,read_based_data가 다릅니다. True일 경우 data average를 기반으로 하고, False일 경우 fitting model average를 기반으로 하는데 0-1V, ~4V라는 fitting quality가 낮은 구역을 제외한 레벨들이기 때문에 서로 섞어써도 별 차이가 없는 것을 알 수 있습니다.
이 부분 역시 레벨 수가 커지면 영향이 생기기는 하겠지만 마찬가지로 variation 영향 속에서 significant하지는 않을 듯 합니다.

In [42]:
scheme_check(volt,r,r_std,r_ref,14,1000,fitting_model_average_choice=True,
             write_based_data=False,read_based_data=False,cycle_variation=True,device_variation=False)

fitting_model_average_choice: 
[mean_a,mean_b,mean_c,mean_d]= [3154737.6, 4.7517561, 2.7140852, 26013.311999999998]
write_based_data:
r= [ 412921.21487398  605076.15483507  863777.33500968 1186003.78396469
 1365053.3276679  1550615.78524327 1737651.98809127 1920960.76244909
 2258087.61584135 2535706.89842316 2746061.2965537  2895611.44096009
 3064044.14273295 3134583.80161944]
read_based_data:
r_ref= [ 508998.68485452  734426.74492238 1024890.55948719 1275528.5558163
 1457834.55645558 1644133.88666727 1829306.37527018 2089524.18914522
 2396897.25713225 2640884.09748843 2820836.3687569  2979827.79184652
 3099313.97217619]
cycle_variation:
meanofstd= [133882.2  122234.07 220532.75 250271.67 237156.48 261253.06 246720.44
 219977.48 211311.81 220284.16 192955.4  171979.44 169466.98 157368.23] 
stdofstd= [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
device_variation:
stdofmean= [0, 0, 0, 0] 
stdofstd= [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[array([     0.        , 412921.21487398, 207838

In [25]:
scheme_check(volt,r,r_std,r_ref,14,1000,fitting_model_average_choice=True,
             write_based_data=False,read_based_data=False,cycle_variation=False,device_variation=True)

fitting_model_average_choice: 
[mean_a,mean_b,mean_c,mean_d]= [3154737.6, 4.7517561, 2.7140852, 26013.311999999998]
write_based_data:
r= [ 412921.21487398  605076.15483507  863777.33500968 1186003.78396469
 1365053.3276679  1550615.78524327 1737651.98809127 1920960.76244909
 2258087.61584135 2535706.89842316 2746061.2965537  2895611.44096009
 3064044.14273295 3134583.80161944]
read_based_data:
r_ref= [ 508998.68485452  734426.74492238 1024890.55948719 1275528.5558163
 1457834.55645558 1644133.88666727 1829306.37527018 2089524.18914522
 2396897.25713225 2640884.09748843 2820836.3687569  2979827.79184652
 3099313.97217619]
cycle_variation:
meanofstd= [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 
stdofstd= [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
device_variation:
stdofmean= [220288.80000000002, 0.8140512000000001, 0.0960402, 19339.372] 
stdofstd= [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[array([     0.        , 412921.21487398, 120735.56513423,      0.        ]), 656.0]
[array([1.00

위의 두 예를 통해 당연하지만 device_variation의 영향이 cycle_variation보다 큰걸 알 수 있습니다

In [14]:
num_level=15
volt,r,r_std,r_ref=load_info(num_level=num_level,choice=0)
scheme_check(volt,r,r_std,r_ref,num_level,1000,fitting_model_average_choice=True,
             write_based_data=False,read_based_data=False,cycle_variation=True,device_variation=True,onecell=False)

I find the information of 15Levels
fitting_model_average_choice: 
[mean_a,mean_b,mean_c,mean_d]= [3154737.6, 4.7517561, 2.7140852, 26013.311999999998]
write_based_data:
r= [  61155.27593001  278285.48206267  501059.64595649  725918.85964533
  954362.47066946 1186003.78396469 1401799.09996076 1625547.27702171
 1848406.84443432 2061673.81390394 2288799.02309421 2510980.5685219
 2727943.59738729 2951529.20292239 3173763.64284554]
read_based_data:
r_ref= [ 169720.37899634  389672.56400958  613489.25280091  840140.66515739
 1070183.12731708 1293901.44196273 1513673.18849123 1736977.06072801
 1955040.32916913 2175236.41849907 2399889.79580806 2619462.08295459
 2839736.40015484 3062646.42288397]
cycle_variation:
meanofstd= [  7687.0293 117162.1    166764.03   160864.4    248972.11   250271.67
 226692.02   295914.44   280031.97   242814.11   248436.77   203030.78
 208887.95   167494.94   167212.72  ] 
stdofstd= [  3843.5146  58581.05    83382.016   80432.2    124486.055  125135.836
 113346.01 

In [65]:
num_level=4
volt,r,r_std,r_ref=load_info(num_level=num_level,choice=1)
scheme_check(volt,r,r_std,r_ref,num_level,1000,fitting_model_average_choice=True,
             write_based_data=False,read_based_data=False,cycle_variation=True,device_variation=True,onecell=False)

I find the information of 4Levels
fitting_model_average_choice: 
[mean_a,mean_b,mean_c,mean_d]= [3154737.6, 4.7517561, 2.7140852, 26013.311999999998]
write_based_data:
r= [  61155.27593001  675530.20830547 2559748.51235896 3161869.59303046]
read_based_data:
r_ref= [ 368342.74211774 1617639.36033222 2860809.05269471]
cycle_variation:
meanofstd= [  7687.0293 163069.6    238056.05   144769.9   ] 
stdofstd= [  3843.5146  81534.8    119028.02    72384.95  ]
device_variation:
stdofmean= [220288.80000000002, 0.8140512000000001, 0.0960402, 19339.372] 
stdofstd= [  3843.5146  81534.8    119028.02    72384.95  ]
35032.13873361276
[array([    0.        , 61155.27593001, 42248.03396414]), 35032.13873361276, 0.0, 997.0]
[997.   3.   0.   0.]
650136.6296896022
[array([1.00000000e+00, 6.75530208e+05, 5.47230296e+05]), 650136.6296896022, 1.0, 841.0]
[153. 841.   6.   0.]
2845695.8542948104
[array([2.00000000e+00, 2.55974851e+06, 3.14890363e+06]), 2845695.8542948104, 3.0, 782.0]
[  0.  17. 782. 201.]
3

## 20180605 cell 수를 늘림에 따라 write quality가 어떻게 변하는지를 알아보자 

준비1: cell을 여러개 다룰 때 필요한 함수가 추가 된다, 이외 기본적 함수는 위의 함수를 그대로 쓴다.

In [19]:
def split_level(level,array_shape, num_cell=None,num_level=None,mode=0):
    assert num_cell!=None and num_level!=None,"Please input num_cell and num_level"
    splited_level = np.zeros(shape=array_shape).transpose()
    level =level.transpose()
    if mode==0:
        quotient=level  #level-1 if level=1,2,3,4,5,6...
        for i in range(num_cell):
            splited_level[num_cell - i - 1,:] = quotient%num_level
            quotient = (quotient - splited_level[num_cell - i - 1, :]) / num_level
    elif mode==1:
        # print("Simplified mode")
        raise NotImplementedError
    else:
        raise ValueError("Error! Please select correct mode.")
    return splited_level.transpose()

def convert_level(splited_level, filter_shape, num_cell=None,num_level=None,mode=0):
    assert num_cell != None and num_level != None, "Please input num_cell and num_level"
    bit_weight=num_level**np.arange(num_cell-1,-1,-1)
    if mode==0:
        converted_level=np.sum(splited_level*bit_weight,axis=-1)
    elif mode==1:
        raise NotImplementedError
    else:
        raise ValueError("Error! Please select correct mode.")
    return converted_level

준비2: 다만 scheme_check함수를 위의 함수에서 약간 바꿔서 쓴다

In [215]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
def scheme_check2(total_level,num_level,num_cell,num_write=1000,
                 Inter_variation_options="False,False,1,False,1",Intra_variation_options="False,False,1,False,1"
                 ,onecell=False):
    #입력 받은 파라미터를 통해 기타 파라미터를 얻는다.
    volt,r,r_std,r_ref=load_info(num_level=num_level,choice=0)
    Inter=eval('['+Inter_variation_options+']')
    Intra=eval('['+Intra_variation_options+']')
    #위에서 얻은 파라미터를 통해 Variation 파라미터를 세팅한다.
    mean_a = 3.156e+06 * 0.9996
    mean_b = 4.463 * 1.0647
    mean_c = 2.713 * 1.0004
    mean_d = -2.509e+04 * 1.0368
    if Intra[0]:
        meanofstd = 0.6 * r_std
        stdofstd = 0.5 * meanofstd
        if Intra[1]:
            print("To be continued..Sorry!")
            exit(0)
        if Intra[3]:
            meanofstd = Intra[4] * meanofstd
            stdofstd = Intra[4] * stdofstd
    else:
        meanofstd = 0. * r_std
        stdofstd = 0. * meanofstd

    if Inter[0]:
        std_a = 3.156e+06 * 0.0698
        std_b = 4.463 * 0.1824
        std_c = 2.713 * 0.0354
        std_d = 2.509e+04 * 0.7708
        # std_a,std_b,std_c,std_d=[0,0,0,0]
        # stdofstd is free, 0일 수도 있고 아닐 수도 있다.
        if Inter[1]:
            print("To be continued..Sorry!")
            exit(0)
        if Inter[3]:
            std_a = Inter[4] * std_a
            std_b = Inter[4] * std_b
            std_c = Inter[4] * std_c
            std_d = Inter[4] * std_d
    else:
        std_a = 0.
        std_b = 0.
        std_c = 0.
        std_d = 0.
        stdofstd = 0. * meanofstd
    print("%10s"%"volt =",volt,"\n%10s"%"r =",r,"\n%10s"%"r_std =",r_std,"\n%10s"%"r_ref =",r_ref)
    length_str=str(len("[mean_a,mean_b,mean_c,mean_d] ="))
    print("\n1.Cell-to-cell Variation params: \n%32s"%"[mean_a,mean_b,mean_c,mean_d] =",
          np.round([mean_a,mean_b,mean_c,mean_d],3),
          "\n%32s"%"[std_a,std_b,std_c,std_d] =",np.round([std_a,std_b,std_c,std_d],3))
    print("\n2.In-cell Variation params:\n%14s"%"meanofstd =",meanofstd,"\n%14s"%"stdofstd =",stdofstd)
    
    #logging용 행렬
    log=np.zeros([total_level,num_write,5])  #level i를 j번째 쓸 때 얻게 되는 5개의 정보를 쓰게 될 행렬
    log_level=np.zeros([total_level,total_level]) #level i를 num_write번 쓸 때 각 레벨이 몇번 나오게 되나를 기록 할 행렬

    cell_per_weight=np.zeros([num_cell,4])
    r_targeted=np.zeros(num_cell)
    r_writed=np.zeros(num_cell)
    level_read=np.zeros(num_cell)
    cell_index=1
    cell_dict={}
    for i in range(total_level):
        splited_level=split_level(np.array([i]),[1,num_cell],num_cell,num_level)
        for j in range(num_write):
            for k in range(num_cell):
                if onecell==False:
                    a_var = np.random.normal(loc=mean_a, scale=std_a)
                    b_var = np.random.normal(loc=mean_b, scale=std_b)
                    c_var = np.random.normal(loc=mean_c, scale=std_c)
                    d_var = np.random.normal(loc=mean_d, scale=std_d)
                    cell_per_weight[k,:]=[a_var,b_var,c_var,d_var]
                elif onecell==True and i==0:
                    a_var = np.random.normal(loc=mean_a, scale=std_a)
                    b_var = np.random.normal(loc=mean_b, scale=std_b)
                    c_var = np.random.normal(loc=mean_c, scale=std_c)
                    d_var = np.random.normal(loc=mean_d, scale=std_d)
                    cell_per_weight[k,:]=[a_var,b_var,c_var,d_var]
                target=int(splited_level[0,k])
                a_var,b_var,c_var,d_var=cell_per_weight[k,:]
                mean_val = a_var/(1+np.exp(-b_var*(volt[target]-c_var))) - d_var
                std_val = np.random.normal(loc=meanofstd[target],scale=stdofstd[target])
                std_val = std_val *(std_val>0) + meanofstd[target]* (std_val<=0)
                r_targeted[k]=mean_a/(1+np.exp(-mean_b*(volt[target]-mean_c))) - mean_d
                r_writed[k] = np.random.normal(loc=mean_val,scale=std_val)
                level_read[k]=make_index(r_writed[k],r_ref)
            converted_level=convert_level(level_read,[1],num_cell,num_level)
            converted_level=np.clip(converted_level,0,total_level-1)
            log[i,j,4]=(i==converted_level)
            log_level[i,int(converted_level)]+=1
        
    #Print summary of the result
        from IPython.display import display_html
        def display_side_by_side(*args):
            html_str=''
            for df in args:
                html_str+=df.to_html()
            display_html(html_str.replace('table','table style="display:inline"'),raw=True)
        table=np.array([[i]+list(*splited_level)+list(r_targeted)+[num_write],
                        [converted_level]+list(level_read)+list(r_writed)+[np.sum(log[i,:,4])]],dtype=np.int32)
        name_columns=[]
        for k in range(num_cell):
            name_columns+=["Cell"+str(k)+"-L"]
        for k in range(num_cell):
            name_columns+=["Cell"+str(k)+"-R"]    
        name_columns=["Level"]+name_columns+["Num_write"]
        table=pd.DataFrame(table,columns=name_columns,index=["Targeted","Real"])

        name_columns=[]
        for k in range(total_level):
            name_columns+=[str(k)]    
        table2=pd.DataFrame(log_level[i,:].reshape([1,total_level]),columns=name_columns,index=["Writed"])
        display_side_by_side(table,table2)
    print('finish')

In [216]:
total_level=16
num_cell=3
num_level = int(0.999999+np.exp(np.log(total_level) / num_cell))
scheme_check2(total_level,num_level,num_cell,num_write=1000,
                 Inter_variation_options="True,False,1,False,1",Intra_variation_options="True,False,1,False,1"
                 ,onecell=False)

I find the information of 3Levels
    volt = [1.77 2.72 4.  ] 
       r = [  61155.277 1625547.1   3173763.8  ] 
   r_std = [ 12811.715 493190.72  278687.84 ] 
   r_ref = [ 843351.2 2399655.5]

1.Cell-to-cell Variation params: 
 [mean_a,mean_b,mean_c,mean_d] = [ 3.1547376e+06  4.7520000e+00  2.7140000e+00 -2.6013312e+04] 
     [std_a,std_b,std_c,std_d] = [2.2028880e+05 8.1400000e-01 9.6000000e-02 1.9339372e+04]

2.In-cell Variation params:
   meanofstd = [  7687.0293 295914.44   167212.72  ] 
    stdofstd = [  3843.5146 147957.22    83606.36  ]


Level 
 Cell0-L 
 Cell1-L 
 Cell2-L 
 Cell0-R 
 Cell1-R 
 Cell2-R 
 Num_write 
 
 
 
 
 Targeted 
 0 
 0 
 0 
 0 
 61155 
 61155 
 61155 
 1000 
 
 
 Real 
 0 
 0 
 0 
 0 
 334151 
 41371 
 64734 
 1000 
 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 6 
 7 
 8 
 9 
 10 
 11 
 12 
 13 
 14 
 15 
 
 
 
 
 Writed 
 1000.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0

Level 
 Cell0-L 
 Cell1-L 
 Cell2-L 
 Cell0-R 
 Cell1-R 
 Cell2-R 
 Num_write 
 
 
 
 
 Targeted 
 1 
 0 
 0 
 1 
 61155 
 61155 
 1625547 
 1000 
 
 
 Real 
 1 
 0 
 0 
 1 
 105083 
 42337 
 1469758 
 896 
 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 6 
 7 
 8 
 9 
 10 
 11 
 12 
 13 
 14 
 15 
 
 
 
 
 Writed 
 53.0 
 896.0 
 51.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0

Level 
 Cell0-L 
 Cell1-L 
 Cell2-L 
 Cell0-R 
 Cell1-R 
 Cell2-R 
 Num_write 
 
 
 
 
 Targeted 
 2 
 0 
 0 
 2 
 61155 
 61155 
 3173763 
 1000 
 
 
 Real 
 2 
 0 
 0 
 2 
 73213 
 109267 
 3346658 
 995 
 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 6 
 7 
 8 
 9 
 10 
 11 
 12 
 13 
 14 
 15 
 
 
 
 
 Writed 
 0.0 
 5.0 
 995.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0

Level 
 Cell0-L 
 Cell1-L 
 Cell2-L 
 Cell0-R 
 Cell1-R 
 Cell2-R 
 Num_write 
 
 
 
 
 Targeted 
 3 
 0 
 1 
 0 
 61155 
 1625547 
 61155 
 1000 
 
 
 Real 
 3 
 0 
 1 
 0 
 14429 
 1036429 
 56469 
 894 
 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 6 
 7 
 8 
 9 
 10 
 11 
 12 
 13 
 14 
 15 
 
 
 
 
 Writed 
 57.0 
 0.0 
 0.0 
 894.0 
 0.0 
 0.0 
 49.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0

...

Level 
 Cell0-L 
 Cell1-L 
 Cell2-L 
 Cell0-R 
 Cell1-R 
 Cell2-R 
 Num_write 
 
 
 
 
 Targeted 
 12 
 1 
 1 
 0 
 1625547 
 1625547 
 61155 
 1000 
 
 
 Real 
 12 
 1 
 1 
 0 
 1920533 
 1064488 
 13598 
 770 
 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 6 
 7 
 8 
 9 
 10 
 11 
 12 
 13 
 14 
 15 
 
 
 
 
 Writed 
 0.0 
 0.0 
 0.0 
 56.0 
 0.0 
 0.0 
 5.0 
 0.0 
 0.0 
 46.0 
 0.0 
 0.0 
 770.0 
 0.0 
 0.0 
 123.0

Level 
 Cell0-L 
 Cell1-L 
 Cell2-L 
 Cell0-R 
 Cell1-R 
 Cell2-R 
 Num_write 
 
 
 
 
 Targeted 
 13 
 1 
 1 
 1 
 1625547 
 1625547 
 1625547 
 1000 
 
 
 Real 
 15 
 1 
 2 
 1 
 2162555 
 2549708 
 1195038 
 694 
 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 6 
 7 
 8 
 9 
 10 
 11 
 12 
 13 
 14 
 15 
 
 
 
 
 Writed 
 0.0 
 2.0 
 0.0 
 1.0 
 47.0 
 3.0 
 0.0 
 5.0 
 0.0 
 2.0 
 41.0 
 3.0 
 38.0 
 694.0 
 37.0 
 127.0

Level 
 Cell0-L 
 Cell1-L 
 Cell2-L 
 Cell0-R 
 Cell1-R 
 Cell2-R 
 Num_write 
 
 
 
 
 Targeted 
 14 
 1 
 1 
 2 
 1625547 
 1625547 
 3173763 
 1000 
 
 
 Real 
 14 
 1 
 1 
 2 
 1909856 
 2188980 
 2820555 
 763 
 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 6 
 7 
 8 
 9 
 10 
 11 
 12 
 13 
 14 
 15 
 
 
 
 
 Writed 
 0.0 
 0.0 
 3.0 
 0.0 
 2.0 
 48.0 
 0.0 
 0.0 
 3.0 
 0.0 
 0.0 
 62.0 
 0.0 
 9.0 
 763.0 
 110.0

Level 
 Cell0-L 
 Cell1-L 
 Cell2-L 
 Cell0-R 
 Cell1-R 
 Cell2-R 
 Num_write 
 
 
 
 
 Targeted 
 15 
 1 
 2 
 0 
 1625547 
 3173763 
 61155 
 1000 
 
 
 Real 
 15 
 1 
 2 
 0 
 1680826 
 3246263 
 44668 
 938 
 
 
 
 
 
 
 0 
 1 
 2 
 3 
 4 
 5 
 6 
 7 
 8 
 9 
 10 
 11 
 12 
 13 
 14 
 15 
 
 
 
 
 Writed 
 0.0 
 0.0 
 0.0 
 1.0 
 0.0 
 0.0 
 52.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 9.0 
 0.0 
 0.0 
 938.0

finish


In [217]:
total_level=4
num_level=total_level
num_cell=1
scheme_check2(total_level,num_level,num_cell,num_write=1000,
                 Inter_variation_options="True,False,1,False,1",Intra_variation_options="True,False,1,False,1"
                 ,onecell=False)

I find the information of 4Levels
    volt = [1.77 2.57 2.86 3.79] 
       r = [  61155.277 1083552.5   2129315.    3161869.5  ] 
   r_std = [ 12811.715 413292.88  358945.7   241283.17 ] 
   r_ref = [ 572353.9 1606433.8 2645592.2]

1.Cell-to-cell Variation params: 
 [mean_a,mean_b,mean_c,mean_d] = [ 3.1547376e+06  4.7520000e+00  2.7140000e+00 -2.6013312e+04] 
     [std_a,std_b,std_c,std_d] = [2.2028880e+05 8.1400000e-01 9.6000000e-02 1.9339372e+04]

2.In-cell Variation params:
   meanofstd = [  7687.0293 247975.73   215367.42   144769.9   ] 
    stdofstd = [  3843.5146 123987.87   107683.71    72384.95  ]


,Level,Cell0-L,Cell0-R,Num_write
Targeted,0,0,61155,1000
Real,0,0,82948,1000
,0,1,2,3
Writed,1000.0,0.0,0.0,0.0


,Level,Cell0-L,Cell0-R,Num_write
Targeted,1,1,1083552,1000
Real,0,0,107956,772
,0,1,2,3
Writed,102.0,772.0,125.0,1.0


,Level,Cell0-L,Cell0-R,Num_write
Targeted,2,2,2129314,1000
Real,2,2,1911047,793
,0,1,2,3
Writed,0.0,103.0,793.0,104.0


,Level,Cell0-L,Cell0-R,Num_write
Targeted,3,3,3161869,1000
Real,3,3,3089888,969
,0,1,2,3
Writed,0.0,0.0,31.0,969.0


finish
